In [1]:
!pip install -q pgmpy

# Noisy-OR model

## Deterministic OR

It is easy to see that an OR between many variables can factorize as a product of tables involving only three variables

$$
\begin{align}\label{eq:or}
 \text{OR}(y|x_1,x_2,x_3) & = \sum_{s\in\{0,1\}} \text{OR}(y|x_3,s)\text{OR}(s|x_1,x_2).
\end{align}
$$

## Noisy OR

The noisy-OR model \[1\] represents a joint probability distribution composed of an effect variable $y$ that has $K$ parents $x_k, k=1,...,K$. For simplicity, we will assume that all of them are binary. It factorizes as

$$
\begin{align}
 P(y=0|x_1,...,x_K) & = P(y=0|L)\prod_{k=1}^{K} P(y=0|x_k)\\
 & = (1-\lambda_0)\prod_{k=1}^{K} (1-\lambda_k)^{x_k},
\end{align}
$$

where $\lambda_k$ is used to parameterize the probability that the parent $k$, if present, could alone determine the effect to have a positive outcome. The parameter $\lambda_0 = P(y=1|L)$, also known as leak probability, is the probability that $y$ occurs by means other than the specified parents.

Note that since $x_k$ appears in the exponent, when the parent is not active ($x_k=0$), the corresponding term is one, so the probability is not affected by $\lambda_k$. On the contrary, when $x_k=1$, the lower the value of $\lambda_k$, the less likely will be that the effect is present, and vice-versa.

## Using exponentially large OR tables
A simple way to represent this model consists of $K$ factors (one for each parent $x_k$) that have a direct relation with a hidden variable $z_k$. These potentials encode the local probability that the effect is active or not, given the value of the cause only:
<table>
    <tr><th>$x_k$</th><th>$z_k$</th><th>$P(z_k|x_k)$</th></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> $1$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> $(1-\lambda_k)$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> $\lambda_k$ </td></tr>
</table>

An additional leak factor $L$ is also introduced to account for the term $(1-\lambda_0)$ and $L$ is usually set to $1$ (i.e. $P(L=1)=1$).

All these factors are combined using a deterministic OR factor. The size of this gate grows exponentially in $K$. For example, for $K=3$:
<table>
    <tr><th> $z_0$ </th> <th> $z_1$ </th> <th> $z_2$ </th> <th> $z_3$ </th> <th> $P(y=0|z_0,z_1,z_2,z_3)$ </th></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 0 </td> <td> 0 </td> <td> $1$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
</table>
<br><center>Table 1: A deterministic OR factor: full table for four inputs</center>

Note that we also need to specify the prior probabilities for $x_k, \forall k$. For simplicity, we will assume they are uniform.

Create a model like that and experiment with different values of $\boldsymbol{\lambda}$. For example, assume $\lambda_0=10^{-3},\lambda_1=0.9,\lambda_2=0.5,\lambda_3=0.99$. The corresponding graphical model is <img src="https://drive.google.com/uc?export=view&id=1xRXh7Yom25kkRraxilSjHeJsl69Vgkx5" width=350 height=230>

Construct this model using python.

In [2]:
# SOLUTION

from pgmpy.factors.discrete import DiscreteFactor
import numpy as np

p = dict()

n_vars = 3
l = [10**(-3), 0.9, 0.5, 0.99]

def noisy_factor(z, x, _lambda):
    """
    Given binary variables x and z, returns p(z|x) according to _lambda.
    """
    return DiscreteFactor(variables=[x, z],
                          cardinality=[2, 2],
                          values=[1, 0, (1-_lambda), _lambda])

# Construct p(z_k|x_k)
p["z0|L"] = noisy_factor("z0", "L", l[0])
p["z1|x1"] = noisy_factor("z1", "x1", l[1])
p["z2|x2"] = noisy_factor("z2", "x2", l[2])
p["z3|x3"] = noisy_factor("z3", "x3", l[3])


# Deterministic OR
p["y|z0,z1,z2,z3"] = DiscreteFactor(variables=["y", "z0", "z1", "z2", "z3"],
                                    cardinality=[2, 2, 2, 2, 2],
                                    values=[1,0,0,0,0,0,0,0,
                                            0,0,0,0,0,0,0,0,
                                            0,1,1,1,1,1,1,1,
                                            1,1,1,1,1,1,1,1])

def prior(x, p0):
    return DiscreteFactor(variables=[x],
                          cardinality=[2],
                          values=[p0, 1-p0])

# Assume uniform priors for p(x_i)
p["x1"] = prior("x1", 0.5)
p["x2"] = prior("x2", 0.5)
p["x3"] = prior("x3", 0.5)
# Assume p(L=1)=1  --> com fariem p[L] = 1 - po, en aquest cas posem p0 = 0 per que sigui p[L] = 1 - 0 = 1 
p["L"] = prior("L", 0)

p["y,x1,x2,x3,L,z0,z1,z2,z3"] = p["y|z0,z1,z2,z3"]*p["z0|L"]*p["z1|x1"]*p["z2|x2"]*p["z3|x3"]*p["x1"]*p["x2"]*p["x3"]*p["L"]
print("Sum of joint prob:", np.sum(p["y,x1,x2,x3,L,z0,z1,z2,z3"].values))

p["y,x1,x2,x3"] = p["y,x1,x2,x3,L,z0,z1,z2,z3"].marginalize(["L", "z0", "z1", "z2", "z3"], inplace=False)
p["y|x1,x2,x3"] = p["y,x1,x2,x3"] / p["y,x1,x2,x3"].marginalize(["y"], inplace=False)
print(p["y|x1,x2,x3"])

Sum of joint prob: 0.9999999999999999
+-------+-------+-------+------+-------------------+
| x3    | x2    | x1    | y    |   phi(x3,x2,x1,y) |
+=======+=======+=======+======+===================+
| x3(0) | x2(0) | x1(0) | y(0) |            0.9990 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(0) | y(1) |            0.0010 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(1) | y(0) |            0.0999 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(1) | y(1) |            0.9001 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(0) | y(0) |            0.4995 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(0) | y(1) |            0.5005 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(1) | y(0) |            0.0500 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(1) | y(1) |            0.9501 |
+-------

Answer the following questions:


1. What is $P_1=P(y=1|x_1=0,x_2=0,x_3=1)$?

**Solution:**
If only parent $x_3$ is ON, we have $P_1=0.9900001$, which is $\lambda_3$.

In [3]:
# SOLUTION

print(p["y|x1,x2,x3"].reduce([("y",1), ("x1",0), ("x2",0), ("x3",1)], inplace=False))

+---------+
|   phi() |
+=========+
|  0.9900 |
+---------+


2. Argue why $P_2=P(y=1|x_1=0,x_2=1,x_3=0)$ is smaller than $P_1$.

**Solution:**
If only parent $x_2$ is ON, we have $P_2=0.500005$. The intuition is right, since $x_2$ has a lower probability of activating $y$ ($\lambda_2<\lambda_3$)

In [4]:
# SOLUTION

print(p["y|x1,x2,x3"].reduce([("y",1), ("x1",0), ("x2",1), ("x3",0)], inplace=False))

+---------+
|   phi() |
+=========+
|  0.5005 |
+---------+


3. Relate $P_3=P(y=1|x_1=0,x_2=1,x_3=1)$ with $P_2$ and $P_1$.

**Solution:**
If both $x_2$ and $x_3$ },\parents are active, $P_3=0.99500005$, i.e., the probability is higher that $P_1$ and $P_2$ individually, because both parents contribute to the activation of $y$, so the intuition is right again.


In [5]:
# SOLUTION

print(p["y|x1,x2,x3"].reduce([("y",1), ("x1",0), ("x2",1), ("x3",1)], inplace=False))

+---------+
|   phi() |
+=========+
|  0.9950 |
+---------+


4. Relate $P_4=P(y=1|x_1=0,x_2=0,x_3=0)$ with all the previous probabilities.

**Solution:**
If no parents are active, then $P_4=10^{-3}$, a very small (but nonzero) probability that $y$ is active.

In [6]:
# SOLUTION

print(p["y|x1,x2,x3"].reduce([("y",1), ("x1",0), ("x2",0), ("x3",0)], inplace=False))

+---------+
|   phi() |
+=========+
|  0.0010 |
+---------+


5. What are the posterior probabilities of each individual parent, if we observe that $y=1$? How do they change if we observe that $y=0$?

In [7]:
# SOLUTION

# Assume uniform prior
p["x1"] = DiscreteFactor(variables=["x1"], cardinality=[2], values=[0.5, 0.5])
p["x2"] = DiscreteFactor(variables=["x2"], cardinality=[2], values=[0.5, 0.5])
p["x3"] = DiscreteFactor(variables=["x3"], cardinality=[2], values=[0.5, 0.5])

p["y,x1,x2,x3"] = p["y|x1,x2,x3"]*p["x1"]*p["x2"]*p["x3"]
print("Sum of values of p(y,x1,x2,x3):", np.sum(p["y,x1,x2,x3"].values)) # Make sure it is a probability distribution
# print(p["y,x1,x2,x3"])

p["y"] =  p["y,x1,x2,x3"].marginalize(["x1", "x2", "x3"], inplace=False)
p["x1|y"] = p["y,x1,x2,x3"].marginalize(["x2", "x3"], inplace=False) / p["y"]
p["x2|y"] = p["y,x1,x2,x3"].marginalize(["x1", "x3"], inplace=False) / p["y"]
p["x3|y"] = p["y,x1,x2,x3"].marginalize(["x1", "x2"], inplace=False) / p["y"]

print("\np(x1|y)\n", p["x1|y"])
print("\np(x2|y)\n", p["x2|y"])
print("\np(x3|y)\n", p["x3|y"])

Sum of values of p(y,x1,x2,x3): 1.0

p(x1|y)
 +------+-------+-------------+
| y    | x1    |   phi(y,x1) |
+======+=======+=============+
| y(0) | x1(0) |      0.9091 |
+------+-------+-------------+
| y(0) | x1(1) |      0.0909 |
+------+-------+-------------+
| y(1) | x1(0) |      0.3925 |
+------+-------+-------------+
| y(1) | x1(1) |      0.6075 |
+------+-------+-------------+

p(x2|y)
 +------+-------+-------------+
| y    | x2    |   phi(y,x2) |
+======+=======+=============+
| y(0) | x2(0) |      0.6667 |
+------+-------+-------------+
| y(0) | x2(1) |      0.3333 |
+------+-------+-------------+
| y(1) | x2(0) |      0.4562 |
+------+-------+-------------+
| y(1) | x2(1) |      0.5438 |
+------+-------+-------------+

p(x3|y)
 +-------+------+-------------+
| x3    | y    |   phi(x3,y) |
+=======+======+=============+
| x3(0) | y(0) |      0.9901 |
+-------+------+-------------+
| x3(0) | y(1) |      0.3712 |
+-------+------+-------------+
| x3(1) | y(0) |      0.0099 |
+---

## Efficient representation

The previous networks contains a factor with exponetial size, which renders it unfeasible for large $K$. Based on the factorization of the deterministic OR shown at the top, can you think of a more efficient way to represent the noisy-OR model?

Answer the following questions:

1. Using the efficient representation, compute the probabilities of the previous subsection and check they are equivalent

In [8]:
# SOLUTION

p_eff = dict()

def deterministicOR(y, x1, x2):
    return DiscreteFactor(variables=[y, x1, x2],
                          cardinality=[2, 2, 2],
                          values=[1, 0, 0, 0, 0, 1, 1, 1])

# Construct p(z_k|x_k), same as before
p_eff["z0|L"] = noisy_factor("z0", "L", l[0])
p_eff["z1|x1"] = noisy_factor("z1", "x1", l[1])
p_eff["z2|x2"] = noisy_factor("z2", "x2", l[2])
p_eff["z3|x3"] = noisy_factor("z3", "x3", l[3])

# Define intermediate factors
p_eff["o1|z1,z2"] = deterministicOR("o1", "z1", "z2")
p_eff["o2|o1,z3"] = deterministicOR("o2", "o1", "z3")
p_eff["y|o2,z0"] = deterministicOR("y", "o2", "z0")

# Same priors as before
p_eff["x1"] = p["x1"]
p_eff["x2"] = p["x2"]
p_eff["x3"] = p["x3"]
p_eff["L"] = p["L"]

p_eff["y,x1,x2,x3,o1,o2,L,z0,z1,z2,z3"] = p_eff["y|o2,z0"]*p_eff["z0|L"]*p_eff["o2|o1,z3"]*p_eff["o1|z1,z2"]*p_eff["z1|x1"]*p_eff["z2|x2"]*p_eff["z3|x3"]*p_eff["x1"]*p_eff["x2"]*p_eff["x3"]*p_eff["L"]
print("Sum of joint prob:", np.sum(p_eff["y,x1,x2,x3,o1,o2,L,z0,z1,z2,z3"].values))

p_eff["y,x1,x2,x3"] =p_eff["y,x1,x2,x3,o1,o2,L,z0,z1,z2,z3"].marginalize(["o1", "o2", "L","z0", "z1", "z2", "z3"], inplace=False)
p_eff["y|x1,x2,x3"] = p_eff["y,x1,x2,x3"] / p_eff["y,x1,x2,x3"].marginalize(["y"], inplace=False)
print(p_eff["y|x1,x2,x3"])

Sum of joint prob: 1.0
+-------+-------+-------+------+-------------------+
| x3    | x2    | x1    | y    |   phi(x3,x2,x1,y) |
+=======+=======+=======+======+===================+
| x3(0) | x2(0) | x1(0) | y(0) |            0.9990 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(0) | y(1) |            0.0010 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(1) | y(0) |            0.0999 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(0) | x1(1) | y(1) |            0.9001 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(0) | y(0) |            0.4995 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(0) | y(1) |            0.5005 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(1) | y(0) |            0.0499 |
+-------+-------+-------+------+-------------------+
| x3(0) | x2(1) | x1(1) | y(1) |            0.9501 |
+-------+-------+------

2. Construct a noisy-OR model with $K=6$ and $\boldsymbol{\lambda} = (10^{-4}, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5)$. Again, assume uniform priors for $x_k$ and $P(L=1)=1$.
    - For $\mathbf{x}=(0,0,0,0,0,1)$, what is the probability $p(y|\mathbf{x})$?
    - Let $\mathbf{x}=(1,0,0,0,0,1)$ What is the probability $p(y|\mathbf{x})$?

In [9]:
p_K6 = dict()
l_K6 = [10**(-4), 0.99, 0.9, 0.8, 0.7, 0.6, 0.5]

# Construct p(z_k|x_k)
p_K6["z0|L"] = noisy_factor("z0", "L", l_K6[0])
p_K6["z1|x1"] = noisy_factor("z1", "x1", l_K6[1])
p_K6["z2|x2"] = noisy_factor("z2", "x2", l_K6[2])
p_K6["z3|x3"] = noisy_factor("z3", "x3", l_K6[3])
p_K6["z4|x4"] = noisy_factor("z4", "x4", l_K6[4])
p_K6["z5|x5"] = noisy_factor("z5", "x5", l_K6[5])
p_K6["z6|x6"] = noisy_factor("z6", "x6", l_K6[6])

# Define intermediate factors
p_K6["o1|z1,z2"] = deterministicOR("o1", "z1", "z2")
p_K6["o2|o1,z3"] = deterministicOR("o2", "o1", "z3")
p_K6["o3|o2,z4"] = deterministicOR("o3", "o2", "z4")
p_K6["o4|o3,z5"] = deterministicOR("o4", "o3", "z5")
p_K6["o5|o4,z6"] = deterministicOR("o5", "o4", "z6")
p_K6["y|o5,z0"] = deterministicOR("y", "o5", "z0")

# Same priors as before
p_K6["L"] = prior("L", 0)
p_K6["x1"] = prior("x1", 0.5)
p_K6["x2"] = prior("x2", 0.5)
p_K6["x3"] = prior("x3", 0.5)
p_K6["x4"] = prior("x4", 0.5)
p_K6["x5"] = prior("x5", 0.5)
p_K6["x6"] = prior("x6", 0.5)

p_K6["joint"] = p_K6["y|o5,z0"]*p_K6["o5|o4,z6"]*p_K6["o4|o3,z5"]*p_K6["o3|o2,z4"]*p_K6["o2|o1,z3"]*p_K6["o1|z1,z2"]\
                *p_K6["z0|L"]*p_K6["z1|x1"]*p_K6["z2|x2"]*p_K6["z3|x3"]*p_K6["z4|x4"]*p_K6["z5|x5"]*p_K6["z6|x6"]\
                *p_K6["L"]*p_K6["x1"]*p_K6["x2"]*p_K6["x3"]*p_K6["x4"]*p_K6["x5"]*p_K6["x6"]
print("Sum of joint prob:", np.sum(p_K6["joint"].values))

p_K6["y,x1,x2,x3,x4,x5,x6"] =p_K6["joint"].marginalize(["o1","o2","o3","o4","o5","L","z0","z1","z2","z3","z4","z5","z6"], inplace=False)
p_K6["y|x1,x2,x3,x4,x5,x6"] = p_K6["y,x1,x2,x3,x4,x5,x6"] / p_K6["y,x1,x2,x3,x4,x5,x6"].marginalize(["y"], inplace=False)
#print("P(y|x1,x2,x3,x4,x5,x6):")
#print(p_K6["y|x1,x2,x3,x4,x5,x6"])

Sum of joint prob: 1.0


In [10]:
print("P(y|x1=0,x2=0,x3=0,x4=0,x5=0,x6=1):")
print(p_K6["y|x1,x2,x3,x4,x5,x6"].reduce([("x1", 0), ("x2", 0), ("x3", 0), ("x4", 0), ("x5", 0), ("x6", 1)], inplace=False))
print("P(y|x1=1,x2=0,x3=0,x4=0,x5=0,x6=1):")
print(p_K6["y|x1,x2,x3,x4,x5,x6"].reduce([("x1", 1), ("x2", 0), ("x3", 0), ("x4", 0), ("x5", 0), ("x6", 1)], inplace=False))

P(y|x1=0,x2=0,x3=0,x4=0,x5=0,x6=1):
+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.5000 |
+------+----------+
| y(1) |   0.5000 |
+------+----------+
P(y|x1=1,x2=0,x3=0,x4=0,x5=0,x6=1):
+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.0050 |
+------+----------+
| y(1) |   0.9950 |
+------+----------+


## References

\[1\]  J. Pearl. Probabilistic reasoning in intelligent systems: networks of plausible inference. Elsevier, 2014.